In [1]:
import pandas as pd
from pprint import pprint
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument, PDFNoOutlines
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTPage, LTChar, LTAnno, LAParams, LTTextBox, LTTextLine

class PDFPageDetailedAggregator(PDFPageAggregator):
    def __init__(self, rsrcmgr, pageno=1, laparams=None):
        PDFPageAggregator.__init__(self, rsrcmgr, pageno=pageno, laparams=laparams)
        self.rows = []
        self.page_number = 0
    def receive_layout(self, ltpage):        
        def render(item, page_number):
            if isinstance(item, LTPage) or isinstance(item, LTTextBox):
                for child in item:
                    render(child, page_number)
            elif isinstance(item, LTTextLine):
                child_str = ''
                for child in item:
                    if isinstance(child, (LTChar, LTAnno)):
                        child_str += child.get_text()
                child_str = ' '.join(child_str.split()).strip()
                if child_str:
                    row = (page_number, item.bbox[0], item.bbox[1], item.bbox[2], item.bbox[3], child_str) # bbox == (x1, y1, x2, y2)
                    self.rows.append(row)
                for child in item:
                    render(child, page_number)
            return
        render(ltpage, self.page_number)
        self.page_number += 1
        self.rows = sorted(self.rows, key = lambda x: (x[0], -x[2]))
        self.result = ltpage

fp = open("A1_SD1_Wortliste_02.pdf", 'rb')
parser = PDFParser(fp)
doc = PDFDocument(parser)
#doc.initialize() # leave empty for no password

rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageDetailedAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

for page in PDFPage.create_pages(doc):
    interpreter.process_page(page)
    # receive the LTPage object for this page
    device.get_result()

In [2]:
DF = pd.DataFrame(device.rows)
DF.columns = ["page","xmin","ymin","xmax","ymax","text"]
alphw = DF[(DF["page"]>7)&(DF["page"]<27)&(DF["ymin"]<760)&(DF["ymin"]>40)&(DF["xmin"]>135)]

In [46]:
import re
otheralp = (DF[DF["page"].isin([3,5,6,7])&(DF["ymin"]<760)&(DF["ymin"]>40)&(DF["xmax"]>135)]["text"]+" ")#.sum().replace("•"," ").replace("/"," ").replace(".","")
otheralp = otheralp.apply(lambda x: re.sub("\d+", " ", x))
otheralp = otheralp.apply(lambda x: ''.join(' ' if ch in ["•","/",".", "="] else ch for ch in x))

In [48]:
otheralp.to_pickle("basicvocabularyA1.pkl")

In [3]:
pd.options.display.max_rows = 3000

In [4]:
def assignblock(x,limits):
    return sum(x>=limits)

def addspace(x):
    output = x.iloc[0]
    for i in range(1,len(x)):
        xi = x.iloc[i]
        if len(output)>=2:
            if (output[-1]=="-") & (output[-2].isalpha()):
                output=output[:-1]+xi
        output+=" "+xi
            
    return output

In [5]:
alphw["columnaux"] = (alphw["xmin"]>230)
alphw["yabs"]=(850-alphw["ymax"]+alphw["page"].apply(int)*2000-15000).apply(lambda x: int(round(x,0)))
entries = alphw[alphw["columnaux"]==0]
entries = entries.sort_values(["page","yabs"])
entries["dy"] = entries["yabs"].diff()
entries["groupaux"] = (entries["dy"]>5).cumsum()
limits = entries.groupby("groupaux").yabs.min()-4
alphw["block"] = alphw["yabs"].apply(lambda x: assignblock(x,limits))
alphw = alphw.sort_values(["columnaux","yabs","xmin"])#.reset_index(drop=True)
final = alphw.groupby(["columnaux","block"]).agg({"text":addspace}).reset_index(drop=False)
entries  = final[final["columnaux"]==0]
examples = final[final["columnaux"]==1]
ff = pd.merge(entries, examples, on="block",how="left")

<ipython-input-5-eaf8a3fe7228>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alphw["columnaux"] = (alphw["xmin"]>230)
<ipython-input-5-eaf8a3fe7228>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alphw["yabs"]=(850-alphw["ymax"]+alphw["page"].apply(int)*2000-15000).apply(lambda x: int(round(x,0)))
<ipython-input-5-eaf8a3fe7228>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [6]:
ff.loc[46,"text_x"] = 'der Arbeitsplatz, -ä, e'
ff.loc[46, "text_y"] = 'An meinem Arbeitsplatz fehlt ein Drucker.'
ff.loc[100,"text_x"] ='der/die Bekannte, -n'
ff.loc[100,"text_y"] = 'Ein Bekannter von mir heißt Klaus.'
ff.loc[206,"text_x"] = 'der Erwachsene, -n'
ff.loc[206,"text_y"] = 'Dieser Film ist nur für Erwachsene.'
ff.loc[219,"text_x"] = 'der Familienname'
ff.loc[219,"text_y"] = 'Meine Familiennamen sind García González.'
ff.loc[298,"text_x"] = "die Großeltern (pl.)"
ff.loc[298,"text_y"] = " Meine Großeltern leben in Japan."
ff.loc[320,"text_x"] = 'die Hausfrau, -en / der Hausmann'
ff.loc[355,"text_x"] = 'der Jugendliche, -n' 
ff.loc[355,'text_y'] = 'Viele Jugendliche kaufen gern ein.'
ff.loc[408,"text_x"] = 'die Lebensmittel (pl.)'
ff.loc[493,"text_x"] = 'der Partner, -/die Partnerin, -nen'
ff.loc[500,"text_x"] = 'die Pommes frites (pl.)'
ff.loc[500,"text_y"] = 'Die Kinder essen Hähnchen mit Pommes frites.'

In [7]:
ff.loc[181,"text_x"] = 'die Ehefrau, -en / der Ehemann, ä, er'

In [10]:
dictionary = ff.dropna()[["text_x","text_y"]]

In [11]:
dictionary.to_pickle("vocabularyA1.pkl")